In [ ]:
import os
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import random
from importlib import reload
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import Callback
from tensorflow.keras.callbacks import ModelCheckpoint, Callback, EarlyStopping

import glob
from shutil import copyfile
from keras.models import load_model

In [ ]:
# Definir las distintas clases
classes = [
    'Class1.1', 'Class1.2', 'Class1.3', 'Class2.1', 'Class2.2', 'Class3.1',
    'Class3.2', 'Class4.1', 'Class4.2', 'Class5.1', 'Class5.2', 'Class5.3',
    'Class5.4', 'Class6.1', 'Class6.2', 'Class7.1', 'Class7.2', 'Class7.3',
    'Class8.1', 'Class8.2', 'Class8.3', 'Class8.4', 'Class8.5', 'Class8.6',
    'Class8.7', 'Class9.1', 'Class9.2', 'Class9.3', 'Class10.1', 'Class10.2',
    'Class10.3', 'Class11.1', 'Class11.2', 'Class11.3', 'Class11.4',
    'Class11.5', 'Class11.6'
]

# Para agregar la extensin a la ID del dataframe
def append_ext(fn):
    return fn + ".jpg"

# Cargar los datos del archivo a un dataframe
traindf = pd.read_csv('./data/original/training_solutions_rev1.csv')

# Cargar los ids de cada imagen
traindf["id"] = traindf['GalaxyID'].astype(str).apply(append_ext)

# Instanciar el data generator
datagen = ImageDataGenerator(
    fill_mode='nearest',
    cval=0,
    rescale=1. / 255,
    rotation_range=90,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    vertical_flip=True,
    validation_split=0.02)

# Se define un datagenerator para el conjunto de datos de entrenamiento
train_generator = datagen.flow_from_dataframe(
    dataframe=traindf,
    directory="./data/original/images_training_rev1",
    x_col="id",
    y_col=classes,
    subset="training",
    batch_size=64,
    seed=123,
    shuffle=True,
    class_mode="other",
    target_size=(224, 224))

# Se define un datagenerator para el conjunto de datos de validacin
valid_generator = datagen.flow_from_dataframe(
    dataframe=traindf,
    directory="./data/original/images_training_rev1",
    x_col="id",
    y_col=classes,
    subset="validation",
    batch_size=64,
    seed=123,
    shuffle=True,
    class_mode="other",
    target_size=(224, 224))

# Pasos necesarios por epoca
STEP_SIZE_TRAIN = train_generator.n // train_generator.batch_size
STEP_SIZE_VALID = valid_generator.n // valid_generator.batch_size

In [ ]:
# Se define el tamaño de la imagen
img_shape = (224, 224, 3)

# Descargar el modelo, descartando la ultima capa
resnet_model = ResNet50(include_top=False, input_shape=img_shape)

In [ ]:
# Se aplica chori a la ultima capa para poder conectarla a una capa densa
# el tamaño es igual al numero de clases, se pasa por una funcion sigmoidal
x = Flatten()(resnet_model.output)
x = Dense(len(classes), activation='sigmoid')(x)

# se ensambla el modelo con los cambios hechos
model = Model(inputs=resnet_model.input, outputs=x)

In [ ]:
# se descongelan todas las capas
for layer in model.layers:
    layer.trainable = True

# se define el optimizador
optimizer = tf.keras.optimizers.Adam(lr=0.001, decay=5e-4)

# se compila el modelo para que emplee el optimizador
model.compile(optimizer, loss='mse', metrics=["accuracy"])

In [ ]:
# Se configuran callbacks para guardar los pesos durante el entrenamiento 

class LossHistory(Callback):
    def on_train_begin(self, logs={}):
        self.losses = []
        self.val_losses = []

    def on_batch_end(self, batch, logs={}):
        self.losses.append(logs.get('loss'))
        self.val_losses.append(logs.get('val_loss'))

# y uno de parada anticipada para evitar el sobreajuste
early_stopping = EarlyStopping(
    monitor='val_loss', patience=4, verbose=1, mode='auto')

history = LossHistory()

from tensorflow.keras.callbacks import ModelCheckpoint
checkpointer = ModelCheckpoint(
    filepath='./models/weights.hdf5', verbose=2, save_best_only=True)

In [ ]:
# se ajustan los pesos del modelo
hist = model.fit_generator(
    train_generator,
    steps_per_epoch=STEP_SIZE_TRAIN,
    validation_data=valid_generator,
    validation_steps=STEP_SIZE_VALID,
    epochs=1,
    callbacks=[history, checkpointer, early_stopping])

In [ ]:
# grafica del costo de entrenamiento y de validacion
plt.figure(figsize=(12, 8))
plt.plot(hist.epoch, hist.history['loss'], label='Training Loss')
plt.plot(
 hist.epoch, hist.history['val_loss'], label='Validation', linestyle='--')
plt.xlabel("Epochs")
plt.ylabel("RMSE")
plt.legend()
plt.show()

In [ ]:
# guardar el modelo
model.save_weights('./models/gaussian.hd5')

# Predicciones

In [ ]:
import cv2
from PIL import Image

In [ ]:
# cargar los pesos
model.load_weights('./models/gaussian.hd5')

In [ ]:
# mostrar la imagen
img = Image.open('/home/julian/Descargas/Gaussian_filter/test/100018.jpg')
img

In [ ]:
# cargar la imagen
img = cv2.imread('/home/julian/Descargas/Gaussian_filter/test/100018.jpg')

In [ ]:
# redimensionar la imagen
img = tf.image.central_crop(img, 0.52830188679)
img = np.expand_dims(img, axis=0)
img.shape

In [ ]:
# realizar una prediccion
y = model.predict(img)
y